#1. Data Load

In [ ]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

# basepath = '/content/drive/MyDrive/DataScience/e-commerce-customer-behavior-analytics'

Mounted at /content/drive


## 1-1. GIT

In [16]:
from getpass import getpass
import os

# Get your GitHub personal access token safely
token = getpass('Enter your GitHub token: ')

# Set your GitHub info
username = 'iamyunaseo'
repo_name = 'e-commerce-customer-behavior-analytics'
repo_url = f"https://{token}@github.com/{username}/{repo_name}.git"
clone_path = '/content/drive/MyDrive/DataScience/e-commerce-customer-behavior-analytics/analytics'

Enter your GitHub token: ··········


In [17]:
!git clone {repo_url} {clone_path}

Cloning into '/content/drive/MyDrive/DataScience/e-commerce-customer-behavior-analytics/analytics'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [23]:
# Navigate into your repo
%cd /content/drive/MyDrive/DataScience/e-commerce-customer-behavior-analytics/analytics

/content/drive/MyDrive/DataScience/e-commerce-customer-behavior-analytics/analytics


In [25]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [26]:
!ls

e-commerce-customer-behavior-analytics.ipynb  README.md


In [27]:
# Set Git configuration
!git config --global user.email "yuna.seo.95@gmail.com"
!git config --global user.name "iamyunaseo"

In [28]:
!git add .
!git commit -m "first commit"
!git push

[main 36910ca] first commit
 1 file changed, 1 insertion(+)
 create mode 100644 e-commerce-customer-behavior-analytics.ipynb
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 2 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 5.31 KiB | 388.00 KiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/iamyunaseo/e-commerce-customer-behavior-analytics.git
   60cdfcd..36910ca  main -> main


## 1-2. Kaggle Load

In [ ]:
# import data from kaggle

from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle datasets download -d carrie1/ecommerce-data

! unzip "ecommerce-data.zip"

Dataset URL: https://www.kaggle.com/datasets/carrie1/ecommerce-data
License(s): unknown
ecommerce-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ecommerce-data.zip
replace data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import libraries

from os import listdir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
os.listdir()

['.config', 'drive', 'ecommerce-data.zip', 'data.csv', 'sample_data']

In [ ]:
df = pd.read_csv('data.csv', encoding='ISO-8859-1')

#2. Data Exploring

let's first see how data looks like.

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
